## Веб-скрейпинг из Википедии и анализ данных на примере страницы с президентами в США

<img src='web_scrap-696x400.jpg' style='width:600px;height:350px'/>

Компания "Тралала" решила проверить навыки работников и поручила одному из них:

1. Сделать веб-скрейпинг с любой веб-страницы(в данном примере - страницы из Википедии);
2. Провести необходимые преобразования данных и убрать лишний 'мусор', который засоряет данные, чтобы можно было их анализировать и проводить стастические действия.
3. С помощью SQLite3 сделать контент-анализ данных и вычислить некоторые статистические признаки.

###### Импортируем необходимые библиотеки

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

Для начала найдем подходящую нам страницу.

Выберем для анализа таблицу с президентами США - https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States

<img src='scrinshot_1.png' style='width:700px;height:400px'/>

Привяжем ссылку к переменной url:

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'

Далее воспользуемся Pandas для того чтобы прочитать страницу и передать ссылку в переменную tables

In [ ]:
tables = pd.read_html(url)

Посмотрим сколько таблиц в было на странице:

In [ ]:
len(tables)

Нужная таблица с презедентами находиться под индексом 0

<img src='scrinshot_2.png' style='width:650px;height:400px'/>

Сохраним таблицу в переменную df_USA_presidents.

In [ ]:
df_USA_presidents = tables[0]

Посмотрим на получившеюся таблицу:

In [ ]:
df_USA_presidents.head()

Получившаяся таблица не особо годиться для использования, так что давайте займемся подготовкой данных к анализу.

Можем заметить что в колонках `Portrait` и `Party[b][15]` у нас проставлены NaN значения. В итоговой таблице нам не понадобяться портреты президентов и цвет их партии, так что применем функцию drop для данных колонок.

In [ ]:
df_USA_presidents = df_USA_presidents.drop(['Portrait', 'Party[b][15]'], axis=1)

In [ ]:
df_USA_presidents.head()

Следующим шагом предлагаю редактировать название колонок, чтобы избавиться от сносок, которые были полезны на html странице, но здесь не имеют ничего подобного.

In [ ]:
df_USA_presidents = df_USA_presidents.rename(columns={'No.[a]': 'No', 
                                                      'Term[14]': 'Term', 
                                                      'Party[b][15].1': 'Party', 
                                                      'Vice President[16]': 'Vice President'})

In [ ]:
df_USA_presidents.head()

Поскольку колонка `No` является номерной от первого президент до настоящего. Сделаем эту колонку индексом в нашем датафрейме.

In [ ]:
df_USA_presidents = df_USA_presidents.set_index('No')

In [ ]:
df_USA_presidents.head()

Колонка с именем и датой рождения/смерти не совсем удобна для того, чтобы проводить анализ. Следующим шагом разделим их отдельно на колонку Name и для начала Birth_Death.

In [ ]:
df_USA_presidents['Name'] = df_USA_presidents['Name(Birth–Death)'].apply(lambda x: x.split('(')[0]) 

df_USA_presidents['Birth–Death'] = df_USA_presidents['Name(Birth–Death)'].\
                                                     apply(lambda x: x.split('(')[1].split(')[')[0])



In [ ]:
df_USA_presidents.head()

In [ ]:
df_USA_presidents.tail()

Как мы можем заметить колонка `Birth–Death` содержит ныне живущих людей, что были на должности президента или сейчас на ней без даты смерти. Чтобы в запросах корректно отображалась информация, запишем для таких президентов значения - 0 и уведомим в характеристиках датафрейма, что в колонке Death_date значение 0 - это отсутствие даты смерти.

In [ ]:
df_USA_presidents['Birth_date'] = df_USA_presidents['Birth–Death'].\
                                                apply(lambda x: x.split('–')[0] if '–' in x else x.split(' ')[1])

df_USA_presidents['Death_date'] = df_USA_presidents['Birth–Death'].\
                                                apply(lambda x: x.split('–')[1] if '–' in x else 0)

In [ ]:
df_USA_presidents.head()

Преобразование колонок мы сделали, теперь удалим исходные, чтобы не было лишних. 

In [ ]:
df_USA_presidents = df_USA_presidents.drop(['Name(Birth–Death)', 'Birth–Death'], axis=1)

Следующим шагом переставим колонки так, как было изначально.

In [ ]:
df_USA_presidents = df_USA_presidents[['Name', 'Birth_date', 'Death_date', 'Term', 'Party', 'Election', 'Vice President']]

In [ ]:
df_USA_presidents.head()

Данные выглядят уже намного лучше, но все еще требуют исправлений. Займемся сроком управления и преобразуем даты в количество дней, для того чтобы было удобно в дальнейшем, если понадобиться, распределять президентов по этому признаку. По скольку нынешний президент все еще на посту, можно было бы использовать сегодняшнюю дату, но ввиду возможных вычислений, лучше будет убрать его и работать только с предыдущими президентами.

In [ ]:
df_USA_presidents['start'] = df_USA_presidents['Term'].apply(lambda date: date.split('–')[0])
df_USA_presidents['end'] = df_USA_presidents['Term'].apply(lambda date: date.split('–')[1])

df_USA_presidents = df_USA_presidents[df_USA_presidents['Name'] != 'Joe Biden']

In [ ]:
df_USA_presidents

Мы разделили даты, но в некоторых из них существуют сноски из html страницы. Поэтому проделаем еще работу для того, чтобы сделать данные чище. 

In [ ]:
df_USA_presidents['start'] = df_USA_presidents['start'].apply(lambda date: date.split('[')[0])
df_USA_presidents['end'] = df_USA_presidents['end'].apply(lambda date: date.split('[')[0])

In [ ]:
df_USA_presidents.head()

Так же сноски из html страницы есть в колонке party. Давайте почистим и эту колонку тоже.

In [ ]:
df_USA_presidents['Party'] = df_USA_presidents['Party'].\
                    apply(lambda party: party.split('[')[0] + party.split('[')[1][2:] if '[' in party else party) 

In [ ]:
df_USA_presidents

Теперь займемся вычислением количества дней.

In [ ]:
df_USA_presidents['start'] = pd.to_datetime(df_USA_presidents['start'])
df_USA_presidents['end'] = pd.to_datetime(df_USA_presidents['end'])


In [ ]:
df_USA_presidents.head()

In [ ]:
df_USA_presidents['Term_in_days'] = (df_USA_presidents['end'] - df_USA_presidents['start']).dt.days

In [ ]:
df_USA_presidents.head()

Мы вычислили срок правления в днях, теперь составим конечный вариант датафрейма. 

Мы будем использовать колонки `Name` - Имя, `Birth_date` - Дата рождения, `Death_date` - Дата смерти, `Term_in_days` - Срок правления в днях, `Party` - Партия. Колонки `Election` и `Vice President` в итоговый датафрейм мы не будем включать, так как в иследовании эти колонки не будут принимать участия.

In [ ]:
df_USA_presidents = df_USA_presidents[['Name', 'Birth_date', 'Death_date', 'Term_in_days', 'Party']]

In [ ]:
df_USA_presidents.head()

Итак мы провели подготовку и очистку данных и теперь можем с ними работать.

#### Краткая информация о датафрейме:

In [ ]:
df_USA_presidents.info()

df_sales — результат запроса. В нём содержится следующая информация:

- Name - ФИО президента
- Birth_date - Дата рождения
- Death_date - Дата смерти (у бывших президентов, которые ныне живы, проставлены в строках 0)
- Term_in_days - Срок правления в днях
- Party - Партия президента

Так же в таблице исключен нынешний президент.

Пропуски не обнаружены. Необходимости преобразовывать типы нет.

Для срока правления в днях устраивает int64 - целые числа. Можно немного уменьшить занимаемую память, понизив тип до int16.


###### Уменьшим использование памяти.

In [ ]:
df_USA_presidents = df_USA_presidents.astype({'Term_in_days' : 'int16'})

In [ ]:
df_USA_presidents.info()

Незначительно уменьшели занимаемую память.

### Перейдем к анализу данных датафрейма с помощью SQLlite3.

Для этого создадим подключение к базе данных и подключим датафрейм к ней.

На выходе получаем количество строк в таблице.

In [ ]:
con = sqlite3.connect('db')
df_USA_presidents_SQL = df_USA_presidents
df_USA_presidents_SQL.to_sql('USA_presidents_SQL',con,index=False,if_exists='replace')

Создадим функцию для того, чтобы удобнее передавать запросы к базе данных.

In [ ]:
def select(sql):
    return pd.read_sql(sql,con)

##### Для начала выведем первых 10-ти президентов из таблицы.

In [ ]:
sql = '''
SELECT 
    * 
FROM 
    USA_presidents_SQL
LIMIT 10
'''

In [ ]:
select(sql)

##### Далее давайте посчитаем сколько президентов от какой партии выступали.

In [ ]:
sql = '''
SELECT 
    Party,
    COUNT(1) as Cnt_party
FROM 
    USA_presidents_SQL
GROUP BY 
    Party
ORDER BY 
    Cnt_party DESC
'''

In [ ]:
select(sql)

 Заметим, что большинство президентов выступали от партий - Republican и Democratic.

#### Взглянем на президента, которые дольше всех находился у правления страной.

In [ ]:
sql = '''
SELECT 
    Name,
    Term_in_days
FROM 
    USA_presidents_SQL
WHERE 
    Term_in_days = (SELECT MAX(Term_in_days) FROM USA_presidents_SQL)
'''

In [ ]:
select(sql)

In [ ]:
4422 / 365.5

В результате запроса мы выяснили, что дольше всех у правления был Franklin D. Roosevelt, который занимал место президента больше 12-ти лет.

#### Теперь посмотрим  на президента, которые меньше всех был у власти.

In [ ]:
sql = '''
SELECT 
    Name,
    Term_in_days
FROM 
    USA_presidents_SQL
WHERE 
    Term_in_days = (SELECT MIN(Term_in_days) FROM USA_presidents_SQL)
'''

In [ ]:
select(sql)

Данный запрос показал, что меньше всех был у власти William Henry Harrison, который умер спустя месяц после выборов.

####  Найдем количество президентов родившихся в 18, 19, 20 веках (Не считая нынешнего).

In [ ]:
sql = '''
SELECT 
    COUNT(Name) FILTER(WHERE Birth_date BETWEEN 1700 and 1799) as '18th_century',
    COUNT(Name) FILTER(WHERE Birth_date BETWEEN 1800 and 1899) as '19th_century',
    COUNT(Name) FILTER(WHERE Birth_date BETWEEN 1900 and 1999) as '20th_century'
FROM 
    USA_presidents_SQL
'''

In [ ]:
select(sql)

По выводу мы можем однозначно можем сказать:

    - 13 президентов родились в 18 веке;
    - 21 президент родились в 19 веке;
    - 11 президентов родились в 20 веке.

#### Вспомним как выглядел наша таблица и посмотрим на президента, который прожил больше других.

In [ ]:
sql = '''
SELECT 
    * 
FROM 
    USA_presidents_SQL
LIMIT 
    5
'''

In [ ]:
select(sql)

In [ ]:
sql = '''
SELECT 
    Name,
    MAX(Death_date - Birth_date) FILTER(WHERE Death_date != 0) as age
FROM 
    USA_presidents_SQL
'''

In [ ]:
select(sql)

Среди ранее живших George H.W. Bush - 41-ый президент США, прожил дольше остальных, но давайте проверим, есть ли кто-то, кто прожил больше лет и все еще живет свою прекрасную жизнь.

In [ ]:
sql = '''
SELECT 
    Name,
    MAX(date('now') - Birth_date) FILTER(WHERE Death_date == 0) - 1 as age
FROM 
    USA_presidents_SQL
'''

In [ ]:
select(sql)

И к большому удивлению запрос вернул Jimmy Carter - 39-го президента США, который прожил на данный момент 98 лет.

## Выводы:

- Из страницы Википедии о презедентах США -  https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States мы достали данные с помощью Pandas и загрузили их в датафрейм.

- Поскольку данные нуждались в очистке и преобразовании, было решено произвести необходимые действия. 
    - Колонка с именем, датой рождения и смерти была разделена на три колонки, для более удобного взаимодействия с данными.
    - Было посчитано из дат о сроке правления, сколько дней пробыл каждый из презедентов на посту.
    - Данные были очищены от сносок, которые были полезны на html странице, но в нашей таблице не несли никакого смысла.
    
- В анализе данных с помощью подготовленного датафрейма был использован SQLite3 и проведены следующие действия:
    - Сколько выступало президентов от каждой партии. Больше всего президентов было от партий - Republican и Democratic.
    - Был найден президент, чей срок правления составлял больше остальных - Franklin D. Roosevelt, который занимал пост президента почти 13 лет. А наименьший срок имел - William Henry Harrison, который умер спустя месяц после выборов.
    - Так же был найден президент, который прожил среди ранее живших больше остальных - George H.W. Bush - 41-ый президент США. А при ныне живущих - Jimmy Carter - 39-ый президент США, который прожил на данный момент 98 лет.
    